## Лабораторная работа No1. Python и Pandas.

In [138]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Задание 1. 

In [139]:
transactions = pd.read_csv('transactions.csv', sep=';', decimal=',')
merchants = pd.read_csv('merchants.csv', sep=';', decimal=',')

In [140]:
merged_df = pd.merge(transactions, merchants, on='merchantid')
# merged_df

In [141]:
merged_df = merged_df[merged_df['fraudbits'] == 0]
# merged_df.info()

In [142]:
guild = merged_df['guild'].value_counts()
print(guild)

guild
Гильдия Обработчиков Шерсти         23337
Ассоциация Оливковых Торговцев      13689
Гильдия Пекарей                     11507
Гильдия Гончаров                    10780
Гильдия Парфюмеров                   9212
Гильдия Августовских Торговцев       8862
Братство Кузнецов                    6820
Братство Виноделов                   3898
Лига Керамистов                      3311
Объединение Фруктовых Садовников     2540
Гильдия Мукомолов                     905
Лига Стеклодувов                      891
Объединение Рыбаков                   775
Союз Судостроителей                   650
Союз Каменотесов                      619
Братство Поставщиков Специй           462
Союз Скотоводов                       315
Общество Ювелиров                     146
Name: count, dtype: int64


In [143]:
# Добавление столбца с амнистией
merged_df['amnesty'] = abs(merged_df['cost'] - merged_df['model_cost']) / merged_df['coef']

merged_df


,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty
0,8315932911717591684,7.39,buy,924-09-27 12:57,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174
1,831575341702446221,22.47,sell,924-09-27 15:48,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825
2,8316302311724840691,9.81,buy,924-09-27 18:51,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605
3,8316277211719765682,6.21,buy,924-09-27 20:05,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238
4,5107559044524868241,41.04,buy,924-09-27 05:55,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,924-09-27 12:55,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783
99995,5924352725499828772,1.21,buy,924-09-27 05:11,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374
99996,4189309751670694249,39.11,sell,924-09-27 20:57,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500
99997,1198308631680924130,22.84,buy,924-09-27 12:56,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407


In [144]:
# Группировка данных по гильдиям для вычисления соотношения продаж к покупкам и средней амнистии
guild_stats = (
    merged_df.groupby('guild')
    .agg(
        sale_count=('countertype', lambda x: (x == 'sell').sum()),
        buy_count=('countertype', lambda x: (x == 'buy').sum()),
        total_amnesty=('amnesty', 'mean'),
    )
    .reset_index()
)

guild_stats

,guild,sale_count,buy_count,total_amnesty
0,Ассоциация Оливковых Торговцев,1330,12359,682.928338
1,Братство Виноделов,392,3506,1726.452617
2,Братство Кузнецов,659,6161,1835.734533
3,Братство Поставщиков Специй,47,415,555.012089
4,Гильдия Августовских Торговцев,849,8013,940.918641
5,Гильдия Гончаров,1019,9761,677.369341
6,Гильдия Мукомолов,78,827,447.359102
7,Гильдия Обработчиков Шерсти,2262,21075,1353.118922
8,Гильдия Парфюмеров,872,8340,926.507174
9,Гильдия Пекарей,1174,10333,1026.286880


In [145]:
# Избегание деления на ноль при расчете соотношения продаж к покупкам
guild_stats['sale_buy_ratio'] = guild_stats['sale_count'] / guild_stats['buy_count'].replace(0, 1)

# Вычисление эффективности гильдий
guild_stats['efficiency'] = guild_stats['sale_buy_ratio'] * guild_stats['total_amnesty']

# Нахождение гильдии с максимальной эффективностью
best_guild = guild_stats.loc[guild_stats['efficiency'].idxmax()]
best_guild_name = best_guild['guild']
best_guild_efficiency = best_guild['efficiency']

print(f"Гильдия с наибольшей эффективностью: {best_guild_name}")
print(f"Эффективность: {best_guild_efficiency:.2f}")

Гильдия с наибольшей эффективностью: Братство Кузнецов
Эффективность: 196.36


In [146]:
# Определение часа с наибольшим количеством продаж для лучшей гильдии
merged_df['date'] = '1' + merged_df['date']
merged_df


,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty
0,8315932911717591684,7.39,buy,1924-09-27 12:57,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174
1,831575341702446221,22.47,sell,1924-09-27 15:48,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825
2,8316302311724840691,9.81,buy,1924-09-27 18:51,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605
3,8316277211719765682,6.21,buy,1924-09-27 20:05,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238
4,5107559044524868241,41.04,buy,1924-09-27 05:55,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,1924-09-27 12:55,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783
99995,5924352725499828772,1.21,buy,1924-09-27 05:11,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374
99996,4189309751670694249,39.11,sell,1924-09-27 20:57,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500
99997,1198308631680924130,22.84,buy,1924-09-27 12:56,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407


In [147]:

# Преобразование столбца date в формат datetime
merged_df['date'] = pd.to_datetime(merged_df['date'], format="%Y-%m-%d %H:%M")
merged_df['hour'] = merged_df['date'].dt.hour

merged_df

,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty,hour
0,8315932911717591684,7.39,buy,1924-09-27 12:57:00,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174,12
1,831575341702446221,22.47,sell,1924-09-27 15:48:00,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825,15
2,8316302311724840691,9.81,buy,1924-09-27 18:51:00,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605,18
3,8316277211719765682,6.21,buy,1924-09-27 20:05:00,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238,20
4,5107559044524868241,41.04,buy,1924-09-27 05:55:00,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,1924-09-27 12:55:00,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783,12
99995,5924352725499828772,1.21,buy,1924-09-27 05:11:00,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374,5
99996,4189309751670694249,39.11,sell,1924-09-27 20:57:00,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500,20
99997,1198308631680924130,22.84,buy,1924-09-27 12:56:00,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407,12


In [148]:
# Фильтрация данных для лучшей гильдии и только продаж
best_guild_sales = merged_df[(merged_df['guild'] == best_guild_name) & (merged_df['countertype'] == 'sell')]

# Группировка данных по часам для подсчета количества продаж
hourly_sales = best_guild_sales.groupby('hour').size().reset_index(name='sales_count')
hourly_sales

,hour,sales_count
0,0,15
1,1,4
2,2,6
3,3,6
4,4,5
5,5,12
6,6,17
7,7,34
8,8,38
9,9,30


In [149]:

# Нахождение часа с наибольшим количеством продаж
best_hour = hourly_sales.loc[hourly_sales['sales_count'].idxmax()]['hour']

print(f"Час с наибольшим количеством продаж для гильдии {best_guild_name}: {best_hour}")

Час с наибольшим количеством продаж для гильдии Братство Кузнецов: 21


### Задание 2.

In [150]:
# Загрузка данных из CSV-файла
data = pd.read_csv('startup_users_visits.csv', parse_dates=['date'], infer_datetime_format=True)


In [151]:
# Шаг 1: Нахождение даты первого визита для каждого пользователя
first_visit = data.groupby('user_id')['date'].min().reset_index()
first_visit.rename(columns={'date': 'first_visit_date'}, inplace=True)

# Объединяем данные с исходным DataFrame
data = data.merge(first_visit, on='user_id')


In [152]:
# Шаг 2: Вычисление количества дней с момента первого визита
data['days_since_first_visit'] = (data['date'] - data['first_visit_date']).dt.days

# Шаг 3: Фильтрация только тех записей, где количество дней <= 90
filtered_data = data[data['days_since_first_visit'] <= 90]

# Шаг 4: Группировка пользователей по месяцу первого визита
filtered_data['first_visit_month'] = filtered_data['first_visit_date'].dt.to_period('M').dt.to_timestamp()

# Фильтрация только до июня 2023 года
filtered_data = filtered_data[filtered_data['first_visit_month'] <= '2023-06-01']


In [153]:

# Подсчет количества оплат для каждого месяца
result = filtered_data[filtered_data['pay'] == 1].groupby('first_visit_month')['user_id'].count().reset_index()
result.rename(columns={'user_id': 'payments_count'}, inplace=True)

# Шаг 5: Формирование итогового CSV-файла
result['first_day_of_month'] = result['first_visit_month'].dt.strftime('%Y-%m-%d')
final_result = result[['first_day_of_month', 'payments_count']]

# Сохранение результата в CSV-файл без заголовка
final_result.to_csv('output.csv', index=False, header=False)

### Задание 3.

In [154]:
# Загрузка данных
data = pd.read_csv('data_fix.txt')


In [155]:
# Исследование данных
print(data.head())
print(data.info())
print(data.describe())


  coupon currency    driver  fact_km     order    plan_km  plan_time  \
0    NaN      RUB  17144870      NaN  34341133   9.012953  10.783333   
1    NaN      RUB  17721342      NaN   7178069   1.518001   6.583333   
2    NaN      RUB  94414125    9.427    846666   9.884320  22.016667   
3    NaN      RUB   7615016   45.956  87372383  44.179568  55.800000   
4    NaN      RUB   1459241   11.256    155631   9.573120  15.800000   

       user    zone  
0  14531103  moscow  
1   8019378  moscow  
2  15232981  moscow  
3  98174968  moscow  
4   7303325  moscow  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17368 entries, 0 to 17367
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   coupon     317 non-null    object 
 1   currency   17368 non-null  object 
 2   driver     17368 non-null  int64  
 3   fact_km    12705 non-null  float64
 4   order      17368 non-null  int64  
 5   plan_km    17368 non-null  float64
 6   pl

In [158]:
unique_values = data['zone'].unique()
print(unique_values)


['moscow' 'podolsk' 'vidnoe' 'ramenskoe' 'krasnogorsk' 'kostroma' 'reutov'
 'himki' 'svo' 'zvenigorod' 'serpukhov' 'odincovo' 'dolgoprudniy'
 'balashiha' 'lyberci' 'troick' 'dme' 'mikhnevo' 'pavlovskyposad'
 'zheleznodorozhny' 'korolev' 'mytishchi' 'chehov' 'pushkino' 'domodedovo'
 'electrostal' 'monino' 'dzerzhinsky' 'lytkarino' 'vko' 'narofominsk'
 'obninsk' 'nakhabino' 'sergievposad' 'stupino' 'kotelniki' 'elektrougli'
 'zelenograd' 'istra' 'dmitrov' 'moscow_region' 'lobnja' 'aprelevka'
 'krasnoznamensk' 'krasnoarmeysk' 'voskresensk' 'klin' 'solnechnogorsk'
 'kaluga_region' 'orehovozuevo' 'chernogolovka' 'tuchkovo' 'mozhaysk'
 'egorievsk' 'chelyabinsk' 'novosibirsk' 'sochi' 'spb']


In [164]:
# Список интересующих нас зон
selected_zones = ['chelyabinsk', 'novosibirsk', 'sochi', 'spb']

# Фильтрация данных по выбранным зонам
drivers_in_selected_zones = data[data['zone'].isin(selected_zones)]['driver'].unique()

# Вывод результатов
if len(drivers_in_selected_zones) > 0:
    print(selected_zones)
    print(drivers_in_selected_zones)


['chelyabinsk', 'novosibirsk', 'sochi', 'spb']
[68102783]


In [159]:
unique_values = data['currency'].unique()
print(unique_values)

['RUB' 'USD']


In [166]:
# Фильтрация данных для художников с currency равным 'usd'
usd_artists = data[data['currency'] == 'USD']['driver'].unique()

# Вывод результатов
if len(usd_artists) > 0:
    print("Художники, использующие валюту USD:")
    print(usd_artists)


Художники, использующие валюту USD:
[87453549]


In [160]:
unique_values = data['coupon'].unique()
print(unique_values)

[nan 'micro_code' 'free_ride']


In [170]:
# Фильтрация данных по промокодам 'micro_code' и 'free_ride'
selected_coupons = ['micro_code', 'free_ride']
filtered_data = data[data['coupon'].isin(selected_coupons)]

# Группировка данных и подсчет использования промокодов каждым художником
coupon_usage = filtered_data.groupby(['coupon', 'driver']).size().reset_index(name='count')

# Поиск художников, использовавших один промокод больше одного раза
suspicious_artists = coupon_usage[coupon_usage['count'] > 10]

# Вывод результатов
if not suspicious_artists.empty:
    print("Художники, использовавшие один промокод более одного раза:")
    print(suspicious_artists[['driver', 'coupon', 'count']])
else:
    print(f"Нет художников, использовавших промокоды {selected_coupons} более одного раза.")

Художники, использовавшие один промокод более одного раза:
     driver     coupon  count
0  39939250  free_ride    234


In [184]:
# Удаляем строки, где fact_km равен 0, чтобы избежать деления на ноль
data = data[data['fact_km'] != 0]

# Вычисляем отношение plan_km к fact_km
data['ratio'] = data['plan_km'] / data['fact_km']

# Определяем порог для "очень большого" отношения
threshold = 10000
suspicious_artists = data[data['ratio'] > threshold]

# Получаем уникальных художников с большим отношением
result = suspicious_artists['driver'].unique()

# Вывод результатов
if len(result) > 0:
    print(f"Художники с очень большим отношением plan_km к fact_km (больше {threshold}):")
    print(result)


Художники с очень большим отношением plan_km к fact_km (больше 10000):
[59986309]


In [191]:

# Группировка по художнику и fact_km, подсчет количества поездок
fact_km_group = data.groupby(['driver', 'fact_km']).size().reset_index(name='trip_count')
suspicious_fact_km = fact_km_group[fact_km_group['trip_count'] > 10]['driver'].unique()
print(suspicious_fact_km)

# Группировка по художнику и plan_km, подсчет количества поездок
plan_km_group = data.groupby(['driver', 'plan_km']).size().reset_index(name='trip_count')
suspicious_plan_km = plan_km_group[plan_km_group['trip_count'] > 10]['driver'].unique()
print(suspicious_plan_km)


[49141855]
[49141855 89859153]


In [198]:
# Группировка по художнику и компаньону, подсчет количества поездок
companion_group = data.groupby(['driver', 'user']).size().reset_index(name='trip_count')

# Фильтрация групп с более чем 10 поездками
suspicious_companions = companion_group[companion_group['trip_count'] > 10]

# Извлечение уникальных художников
result = suspicious_companions['driver'].unique()

# Вывод результатов
if len(result) > 0:
    print("Художники с более чем 10 поездками с одним и тем же компаньоном:")
    print(result)


Художники с более чем 10 поездками с одним и тем же компаньоном:
[ 2195293  9871171 10832805 11213429 32692269 36098567 66809212 89859153
 90985908]


In [201]:
data[data['driver'] == 90985908].head(100)

,coupon,currency,driver,fact_km,order,plan_km,plan_time,user,zone,diff_km,diff_time,ratio
16655,NaN,RUB,90985908,5.129,30320833,6.805875,14.483333,95305689,lyberci,-1.676875,0.354131,1.326940
16656,NaN,RUB,90985908,7.280,28113089,4.337507,13.833333,95305689,lyberci,2.942493,0.526265,0.595811
16657,NaN,RUB,90985908,7.035,64728765,3.772592,14.966667,95305689,lyberci,3.262408,0.470045,0.536260
16658,NaN,RUB,90985908,5.134,65308234,4.182236,12.800000,95305689,lyberci,0.951764,0.401094,0.814616
16659,NaN,RUB,90985908,4.269,85956196,4.276451,10.383333,95305689,kotelniki,-0.007451,0.411140,1.001745
16660,NaN,RUB,90985908,1.719,75451378,6.482285,17.083333,95305689,moscow,-4.763285,0.100624,3.770963
16661,NaN,RUB,90985908,6.959,2675845,7.937331,21.983333,95305689,lyberci,-0.978331,0.316558,1.140585
16662,NaN,RUB,90985908,1.068,54432541,1.057062,6.083333,95305689,lyberci,0.010938,0.175562,0.989759
16663,NaN,RUB,90985908,1.803,6183432,3.086259,8.333333,95305689,lyberci,-1.283259,0.216360,1.711735
16664,NaN,RUB,90985908,1.451,333682,2.903161,9.783333,95305689,lyberci,-1.452161,0.148313,2.000800


Подозрительные художники: 2195293, 68102783, 39939250, 59986309, 49141855, 89859153, 90985908